In [1]:
import sys

sys.path.append("../berries")


In [2]:
from datasets import load_dataset

mnist = load_dataset("mnist").with_format("jax")
mnistData = mnist['train']
X_img = mnistData['image']
y = mnistData['label']
X_img_test = mnist["test"]["image"]
n_test_samples = X_img_test.shape[0]
X_test = X_img_test.reshape((n_test_samples, -1))
y_test = mnist["test"]["label"]
n_samples, _, _ = X_img.shape
X = X_img.reshape((n_samples, -1))
n_samples, d_in = X.shape
d_out = len(set(y.tolist()))

/home/owner/Projects/picojam/pico/.venv/lib/python3.10/site-packages/datasets/load.py:1429: FutureWarning: The repository for mnist contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/mnist
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [3]:
d_h_layer = 128
d_h = 64
n_layers = 1


In [11]:
import optax
from jax.numpy import mean
from jax import grad, jit
from jax.tree_util import tree_map
from pf import F, _
# import nn
# import importlib
# importlib.reload(nn)
import random_utils
from nn import mglu_config, mglu, init_weights, fmt_weights

seed = 0
key_gen = random_utils.infinite_safe_keys(seed)

def accuracy(logits, y):
    return (logits.argmax(-1) == y).mean()


mglu_b = F(mglu).vmap((0, None, None), 0)

def sce_loss(to_logits, x, y, W):
    return optax.softmax_cross_entropy_with_integer_labels(to_logits(x, W), y)




loss_b_all = F(sce_loss).f(mglu, _, _, _).vmap((0, 0, None), 0)
loss_d = loss_b_all.f(X, y, _)
loss_b_d = loss_d >> mean

lr = 0.001
mask_fn = lambda p: tree_map(lambda x: not isinstance(x, int), p)
# opt = optax.multi_transform({"sgd": optax.rmsprop(lr), "zero": optax.set_to_zero()}, mask_fn)
opt = optax.masked(optax.rmsprop(lr), mask_fn)


method = {"type": "zer0", "std": 0.001}
# method = {"type": "normal", "std": 0.01}
W = init_weights(next(key_gen), mglu_config(d_in, d_h_layer, d_out, d_h, n_layers, method))
loss0 = loss_b_d(W)
print(loss0)
print(fmt_weights(W)[0])
state = opt.init(W)

@jit
def update(W, opt_state):
    grads = grad(loss_b_d)(W)
    updates, opt_state = opt.update(grads, opt_state)
    new_W = optax.apply_updates(W, updates)
    return new_W, opt_state

3.1588132
tuple:
    sglu:
        wv:
            array shape: (784, 64)
        wu:
            array shape: (784, 64)
        wo:
            array shape: (64, 128)
        total params: 108544
    rmsn:
        d:
            128.0
        total params: 1
    total params: 108545
tuple:
    sglu:
        wv:
            array shape: (128, 64)
        wu:
            array shape: (128, 64)
        wo:
            array shape: (64, 10)
        total params: 17024
    rmsn:
        d:
            10.0
        total params: 1
    total params: 17025
total params: 125570



In [12]:
def get_accuracy(x, y, W):
    return accuracy(mglu(x, W), y)

get_accuracy_b = F(get_accuracy).vmap(in_axes=(0, 0, None), out_axes=0)
get_accuracy_b_d = get_accuracy_b.f(X, y, _) >> mean
get_accuracy_b_t = get_accuracy_b.f(X_test, y_test, _) >> mean

for i in range(1000):
    W, state = update(W, state)
    if i % 50 == 0:
        print(get_accuracy_b_d(W), get_accuracy_b_t(W), loss_b_d(W))
print(loss_d(W))

0.09871667 0.098 2.9003065
0.9199167 0.923 0.66302013
0.9577 0.95239997 0.43886253
0.97125 0.9625 0.37138352
0.97688335 0.96819997 0.35107404
0.9789 0.9691 0.34038138
0.98256665 0.9693 0.32953045
0.9867833 0.97249997 0.31877425
0.98845 0.97389996 0.31075525
0.99121666 0.9736 0.3010577
0.98918337 0.9719 0.30412602
0.9899833 0.96999997 0.3012402
0.9920667 0.9711 0.2938076
0.9943 0.9741 0.2851047
0.99365 0.97309995 0.28448004
0.99635 0.974 0.27580982
0.9953167 0.9738 0.27615047
0.9942167 0.97239995 0.27758375
0.99805003 0.9745 0.26383737
0.99738336 0.9729 0.265202
[0.25933945 0.24239717 0.25948963 ... 0.24418825 0.2602984  0.25811207]
